## Two body problem

We use Euler-Lagrange equation to solve this problem

For two body sysetm, we set that:


Planet 1: mass $m_1$, position: $(x_1, y_1)$, Planet 2: mass $m_2$, position: $(x_2, y_2)$


So, The Euler-Lagrange equation should be:

$\begin{align}
 \mathcal{L} = \frac12 m_1 (\dot x_1^2 + \dot y_1^2) + \frac12 m_2 (\dot x_2^2 + \dot y_2^2) + \frac{G m_1 m_2}{r}
  \;
\end{align}$



note that $r = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$



Solving Euler Euler-Lagrange equation, we have:

$\begin{align}
 \ddot x_1 = \frac{G m_2 (x_2-x_1)}{r^{3}}
\end{align}$, $\begin{align}
 \ddot y_1 = \frac{G m_2 (y_2-y_1)}{r^{3}}
\end{align}$

$\begin{align}
 \ddot x_2 = \frac{G m_1 (x_1-x_2)}{r^{3}}
\end{align}$, $\begin{align}
 \ddot y_2 = \frac{G m_1 (y_1-y_2)}{r^{3}}
\end{align}$

We set up a vector to include all these equations

$
\begin{align}
\vec{y} = \begin{pmatrix} x_1 (t) & \dot x_1(t) & y_1(t)& \dot y_1(t) & x_2 (t) & \dot x_2(t) & y_2(t)& \dot y_2(t)\end{pmatrix}^{T}
\quad
\end{align}$

$\begin{align}
\frac{d\vec{y}}{dt}  = \begin{pmatrix} \dot x_1 (t) & \ddot x_1(t) & \dot y_1(t)& \ddot y_1(t) & \dot x_2 (t) & \ddot x_2(t) & \dot y_2(t)& \ddot y_2(t) \end{pmatrix}^{T}
\\ \end{align}
$

$\begin{align}
    =\begin{pmatrix}\dot x_1 (t) & -\frac{m_2(x_1-x_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{3/2}} & \dot y_1 (t)&-\frac{m_2(y_1-y_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{3/2}} & \dot x_2 (t) & \frac{m_1(x_1-x_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{3/2}} & \dot y_2 (t) & \frac{m_1(y_1-y_2)}{((x_1-x_2)^2+(y_1-y_2)^2)^{3/2}} \end{pmatrix}^{T}
\end{align}
$

In [ ]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import matplotlib

In [ ]:
font_size = 10
plt.rcParams.update({'font.size': font_size})

In [ ]:
class GravitationalOrbist:
    """
    This class implements the parameters and Larange's equations for two particles orbiting
    under gravitational attraction.
    
    Parameters
    ----------
    m_1 : 
        type:float
        Physical meaning: mass
    m_2 : 
        type:float
        Physical meaning: mass
    G : 
        type:float
        Physical meaning: G

    Methods
    ----------
    dy_dt(t, y)
        This function returns the right-hand side of the diffeq: 
        [dy/dt d^2y/dt^2]
    """
    
    def __init__(self,m_1=1.,m_2=1.,G=1.):
        self.m_1 = m_1
        self.m_2 = m_2
        self.G = G
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dy/dt d^2y/dt^2]
        
        Parameters
        ----------
        t : 
            type: float
            Physical meaning: time 
        y : 
            type: float
            Physical meaning: 8-component vector with
                    y[0] = x_1(t) and y[1] = x_dot_1(t)
                    y[2] = y_1(t) and y[3] = y_dot_1(t)
                    y[4] = x_2(t) and y[5] = x_dot_2(t)
                    y[6] = y_2(t) and y[7] = y_dot_2(t)

                    
            
        """
        
        # Denominator that shows up in all of the Lagrange equations
        r_12 = ((y[0] - y[4])**2 + (y[2] - y[6])**2)**(1./2.)
        
        x_dot_1 = y[1]
        y_dot_1 = y[3]
        x_dot_2 = y[5]
        y_dot_2 = y[7]
        
        # Lagrange's equations
        x_dot_dot_1 = -self.G * self.m_2 * (y[0]-y[4]) / r_12**3
        y_dot_dot_1 = -self.G * self.m_2 * (y[2]-y[6]) / r_12**3
        x_dot_dot_2 = self.G * self.m_1 * (y[0]-y[4]) / r_12**3
        y_dot_dot_2 = self.G * self.m_1 * (y[2]-y[6]) / r_12**3
        
        return x_dot_1, x_dot_dot_1, y_dot_1, y_dot_dot_1, \
               x_dot_2, x_dot_dot_2, y_dot_2, y_dot_dot_2
    
    
    def solve_ode(self, t_pts, x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, 
                  x_2_0, x_dot_2_0, y_2_0, y_dot_2_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]  
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = solution.y
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2
    
    def solve_ode_Leapfrog(self, t_pts, x_1_0, x_dot_1_0, y_1_0, 
                           y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0):
        """
        Solve the ODE given initial conditions using the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for x1, x1_dot, x1_dot_half, with zeros
        # and similarly for the other 3 variables y1, x2, and y2
        num_t_pts = len(t_pts)
        
        x_1 = np.zeros(num_t_pts)
        x_dot_1 = np.zeros(num_t_pts)
        x_dot_1_half = np.zeros(num_t_pts)
        
        y_1 = np.zeros(num_t_pts)
        y_dot_1 = np.zeros(num_t_pts)
        y_dot_1_half = np.zeros(num_t_pts)
        
        x_2 = np.zeros(num_t_pts)
        x_dot_2 = np.zeros(num_t_pts)
        x_dot_2_half = np.zeros(num_t_pts)
        
        y_2 = np.zeros(num_t_pts)
        y_dot_2 = np.zeros(num_t_pts)
        y_dot_2_half = np.zeros(num_t_pts)
        
        # initial conditions
        x_1[0] = x_1_0
        x_dot_1[0] = x_dot_1_0
        
        y_1[0] = y_1_0
        y_dot_1[0] = y_dot_1_0
        
        x_2[0] = x_2_0
        x_dot_2[0] = x_dot_2_0
        
        y_2[0] = y_2_0
        y_dot_2[0] = y_dot_2_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            
            # first-half action
            y = [x_1[i], x_dot_1[i], y_1[i], y_dot_1[i], x_2[i], x_dot_2[i], y_2[i], y_dot_2[i]]
            
            x_dot_1_half[i] = x_dot_1[i] + self.dy_dt(t, y)[1] * delta_t/2.
            x_1[i+1] = x_1[i] + x_dot_1_half[i] * delta_t
            
            y_dot_1_half[i] = y_dot_1[i] + self.dy_dt(t, y)[3] * delta_t/2.
            y_1[i+1] = y_1[i] + y_dot_1_half[i] * delta_t
            
            x_dot_2_half[i] = x_dot_2[i] + self.dy_dt(t, y)[5] * delta_t/2.
            x_2[i+1] = x_2[i] + x_dot_2_half[i] * delta_t
            
            y_dot_2_half[i] = y_dot_2[i] + self.dy_dt(t, y)[7] * delta_t/2.
            y_2[i+1] = y_2[i] + y_dot_2_half[i] * delta_t
            
            # second-half action
            y = [x_1[i+1], x_dot_1[i], y_1[i+1], y_dot_1[i], x_2[i+1], x_dot_2[i], y_2[i+1], y_dot_2[i]]
            
            x_dot_1[i+1] = x_dot_1_half[i] + self.dy_dt(t, y)[1] * delta_t/2.
            
            y_dot_1[i+1] = y_dot_1_half[i] + self.dy_dt(t, y)[3] * delta_t/2.
            
            x_dot_2[i+1] = x_dot_2_half[i] + self.dy_dt(t, y)[5] * delta_t/2.
            
            y_dot_2[i+1] = y_dot_2_half[i] + self.dy_dt(t, y)[7] * delta_t/2.
            
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2
    
    def energy(self, x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2):
        """
        This function calculates the energy given by position and velocity 
        """

        
        return 1./2. * ( self.m_1 * (x_dot_1**2 + y_dot_1**2) + self.m_2 * (x_dot_2**2 + y_dot_2**2)) - self.G * self.m_1 * self.m_2 * ((x_1-x_2)**2 + (y_1-y_2)**2)**(-1./2.)


In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts - plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts - plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

# Setting initial value
m_1 = 10.
m_2 = 9.
m_3 = 1.    #we need m4 >> m3 
m_4 = 80. 
G = 1.
o1 = GravitationalOrbist(m_1,m_2,G)
o2 = GravitationalOrbist(m_3,m_4,G)

# Setting time 
t_start = 0.
t_end = 100.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  
start, stop = start_stop_indices(t_pts, t_start, 100.)


# Initial conditions
x_1_0, x_dot_1_0 = -1., 1
y_1_0, y_dot_1_0 = 1., 1.
x_2_0, x_dot_2_0 = -(m_1 / m_2) * x_1_0, -(m_1 / m_2) * x_dot_1_0
y_2_0, y_dot_2_0 = -(m_1 / m_2) * y_1_0, -(m_1 / m_2) * y_dot_1_0

x_3_0, x_dot_3_0 = -1., 1.
y_3_0, y_dot_3_0 = 1., 1.
x_4_0, x_dot_4_0 = -(m_3 / m_4) * x_3_0, -(m_3 / m_4) * x_dot_3_0
y_4_0, y_dot_4_0 = -(m_3 / m_4) * y_3_0, -(m_3 / m_4) * y_dot_3_0

x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = o1.solve_ode(t_pts, x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0)
x_3, x_dot_3, y_3, y_dot_3, x_4, x_dot_4, y_4, y_dot_4 = o2.solve_ode(t_pts, x_3_0, x_dot_3_0, y_3_0, y_dot_3_0, x_4_0, x_dot_4_0, y_4_0, y_dot_4_0)


#Now let's find the energy difference of the whole system (SciPy ODE)
E_t = o1.energy(x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2)
E_0 = E_t[0]
E_Diff = np.fabs((E_t - E_0) / E_0)

#Now solve equations using the Leapfrog
x_1_L, x_dot_1_L, y_1_L, y_dot_1_L, x_2_L, x_dot_2_L, y_2_L, y_dot_2_L = o1.solve_ode_Leapfrog(t_pts, x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0)

#Now let's find the energy difference of the whole system (Leapfrog)
E_pts_L = o1.energy(x_1_L, x_dot_1_L, y_1_L, y_dot_1_L, x_2_L, x_dot_2_L, y_2_L, y_dot_2_L)
E_0_L = E_pts_L[0]
E_Diff_L = np.fabs((E_pts_L - E_0) / E_0)


#### Everything is in place, now let's create the graphs

In [ ]:
fig_4 = plt.figure(figsize=(16,8))

overall_title = 'SciPy ODE: m_1 = 10, m_2 = 9, m_3 = 1, m_4 = 80.'
fig_4.suptitle(overall_title, va='baseline')

ax_4a = fig_4.add_subplot(1,2,1)
ax_4a.plot(x_1, y_1, color='blue', label='m_1' )
ax_4a.plot(x_2, y_2, color='red', label='m_2')
ax_4a.set_ylabel('y')
ax_4a.set_xlabel('x')
ax_4a.set_title('orbit')
ax_4a.legend()

ax_4b = fig_4.add_subplot(1,2,2)
ax_4b.plot(x_3, y_3, color='blue', label='m_3')
ax_4b.plot(x_4, y_4, color='red', label='m_4')
ax_4b.set_ylabel('y')
ax_4b.set_xlabel('x')
ax_4b.set_title('orbit(m4>>m3)')
ax_4b.legend()

#### Now let's prove that Leapfrog is more energy-efficient than SciPy ODE

In [ ]:
fig_4 = plt.figure(figsize=(16,8))

overall_title = 'm_1 =10, m_2 = 9,  G = 1'
fig_4.suptitle(overall_title, va='baseline')

ax_4a = fig_4.add_subplot(1,2,1)
ax_4a.plot(x_1_L, y_1_L, color='blue', label='m_1')
ax_4a.plot(x_2_L, y_2_L, color='red', label='m_2')
ax_4a.set_ylabel('y')
ax_4a.set_xlabel('x')
ax_4a.set_title('orbit(Leapfrog)')
ax_4a.legend()

ax_4b = fig_4.add_subplot(1,2,2)
ax_4b.plot(t_pts[start : stop], E_Diff_L[start : stop], color='blue', label='Leapfrog')
ax_4b.plot(t_pts[start : stop], E_Diff[start : stop], color='red', label='SciPy ODE')
ax_4b.set_ylabel('Energy(J)')
ax_4b.set_xlabel('Time(s)')
ax_4b.set_title('Energy Error')
ax_4b.legend()